In [70]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http import models
import openai
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxx"

In [71]:
qdrant_client = QdrantClient(
    host="<HOSTNAME>".
    api_key="<API_KEY>",
)

In [73]:
qdrant_client.recreate_collection(
    collection_name="mycollection",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)
print("Create collection reponse:", qdrant_client)

Create collection reponse: <qdrant_client.qdrant_client.QdrantClient object at 0x0000023B216C2F90>


In [74]:
collection_info = qdrant_client.get_collection(collection_name="mycollection")

print("Collection info:", collection_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0)) payload_schema={}


## Read PDF file

In [75]:
import pdfplumber

fulltext = ""
with pdfplumber.open("starship.pdf") as pdf:
    # loop over all the pages
    for page in pdf.pages:
        fulltext += page.extract_text()

# print(fulltext)


## Chunk PDF text

In [76]:

text = fulltext

chunks = []
while len(text) > 500:
    last_period_index = text[:500].rfind('.')
    if last_period_index == -1:
        last_period_index = 500
    chunks.append(text[:last_period_index])
    text = text[last_period_index+1:]
chunks.append(text)


for chunk in chunks:
    print(chunk)
    print("---")

STARSHIP
USERS
GUIDE
Revision 1.0 | March 2020
COPYRIGHT
Subject to the existing rights of third parties, Space Exploration Technologies Corp. (SpaceX) is the owner of the copyright in this
work, and no portion hereof is to be copied, reproduced, or disseminated without the prior written consent of SpaceX.
© Space Exploration Technologies Corp. All rights reserved
---
STARSHIP PAYLOAD GUIDE
COMPANY DESCRIPTION
SpaceX was founded in 2002 to revolutionize access
to space and enable a multi-planetary society. Today,
SpaceX performs routine missions to space with its
Falcon 9 and Falcon Heavy launch vehicles for a
diverse set of customers, including the National
Aeronautics and Space Administration (NASA), the
Department of Defense, international governments,
and leading commercial companies
---
 SpaceX provides
further support to NASA with the Dragon spacecraft by
conducting cargo resupply and return missions to and
from the International Space Station (ISS). Soon,
SpaceX will begin trans

## Create embeddings and index with qdrant

In [77]:
from qdrant_client.http.models import PointStruct

points = []
i = 1
for chunk in chunks:
    i += 1

    print("Embeddings chunk:", chunk)
    response = openai.Embedding.create(
        input=chunk,
        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']

    points.append(PointStruct(id=i, vector=embeddings, payload={"text": chunk}))
    

Embeddings chunk: STARSHIP
USERS
GUIDE
Revision 1.0 | March 2020
COPYRIGHT
Subject to the existing rights of third parties, Space Exploration Technologies Corp. (SpaceX) is the owner of the copyright in this
work, and no portion hereof is to be copied, reproduced, or disseminated without the prior written consent of SpaceX.
© Space Exploration Technologies Corp. All rights reserved
Embeddings chunk: STARSHIP PAYLOAD GUIDE
COMPANY DESCRIPTION
SpaceX was founded in 2002 to revolutionize access
to space and enable a multi-planetary society. Today,
SpaceX performs routine missions to space with its
Falcon 9 and Falcon Heavy launch vehicles for a
diverse set of customers, including the National
Aeronautics and Space Administration (NASA), the
Department of Defense, international governments,
and leading commercial companies
Embeddings chunk:  SpaceX provides
further support to NASA with the Dragon spacecraft by
conducting cargo resupply and return missions to and
from the International Spac

In [78]:

operation_info = qdrant_client.upsert(
    collection_name="mycollection",
    wait=True,
    points=points
)

print("Operation info:", operation_info)

Operation info: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


## final function to put everything together (query qdrant)

In [92]:
def create_answer_with_context(query):
    response = openai.Embedding.create(
        input="What is starship?",
        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']

    search_result = qdrant_client.search(
        collection_name="mycollection",
        query_vector=embeddings, 
        limit=5
    )

    prompt = "Context:\n"
    for result in search_result:
        prompt += result.payload['text'] + "\n---\n"
    prompt += "Question:" + query + "\n---\n" + "Answer:"

    print("----PROMPT START----")
    print(":", prompt)
    print("----PROMPT END----")

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
        )

    return completion.choices[0].message.content

    

## Create answer with context

In [93]:

input = "what is starship"
answer = create_answer_with_context(input)
print(answer)


----PROMPT START----
: Context:
 The Starship Program now
leverages SpaceX’s experience to introduce a next-
generation, super heavy-lift space transportation
system capable of rapid and reliable reuse.
STARSHIP PROGRAM OVERVIEW
SpaceX’s Starship system represents a fully reusable
transportation system designed to service Earth orbit
needs as well as missions to the Moon and Mars
---
 This
two-stage vehicle—composed of the Super Heavy
rocket (booster) and Starship (spacecraft) as shown in
Figure 1—is powered by sub-cooled methane and
oxygen. Starship is designed to evolve rapidly to meet
near term and future customer needs while maintaining
the highest level of reliability.
STARSHIP USERS GUIDE
Starship has the capability to transport satellites,
payloads, crew, and cargo to a variety of orbits and
Earth, Lunar, or Martian landing sites
---
 4STARSHIP PAYLOAD GUIDE
PERFORMANCE CREW CONFIGURATION
The Starship and Super Heavy system offers SpaceX was founded with the goal of making life 